In [1]:
from rag import RAG
from tqdm import tqdm

model = RAG()

Typesense container (presumably) exists already.
Typesense container is already running...


2024-04-14 13:07:15.548957: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-14 13:07:15.570844: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-14 13:07:15.995609: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
config = {
    "search_type": "typesense",
    "use_classifier": True,
    "n_results": 5,
    "model_kwargs": {"temperature": 0.0},
}

In [3]:
questions = [
    "Is there a math major at SUNY Brockport?",
    "Is there an engineering major at SUNY Brockport?",
    "Tell me about the nursing major.",
    "How can I get involved in undergraduate research?",
    "How can I get involved in clubs?",
    "What options are there for financial aid at Brockport?",
    "How can I report sexual misconduct?",
    "Can I get credit for completing an internship in the math department?",
    "Is there a gym available to students on campus?",
    "How can I apply to SUNY Brockport?",
    "What is the meaning of life?",
]

output = []

for question in tqdm(questions):
    output.append(model(question=question, config=config))

output

100%|██████████| 11/11 [00:15<00:00,  1.37s/it]


['Yes, there is a math major at SUNY Brockport.',
 "I'm sorry, I don't know.",
 'The Nursing Major at SUNY Brockport provides students with the knowledge, behaviors, and skills needed for nursing practice at the baccalaureate level. The program emphasizes critical assessment, diagnosis, and treatment skills through clinical rotations. Upon acceptance, students transition from "Nursing Intent" to Nursing Major status, receiving guidance in transfer credits, advisement, and registration. Advanced Practice Nurses are in high demand in major health systems and community organizations in the region. Admission to the DNP program considers academic requirements and professional demands for a systems leadership role.',
 'You can get involved in undergraduate research at SUNY Brockport by working closely with a faculty mentor on scholarly or creative projects related to your major. Reach out to the Scholarship, Research & Sponsored Programs Office for more information and opportunities availabl

# old

In [1]:
import pandas as pd
from datasets import load_dataset

qas = load_dataset("msaad02/brockport-gpt-4-qa")['test'].to_pandas()
data = pd.read_csv("../data_collection/data/chunked_data.csv")

In [2]:
len(qas)

1345

In [2]:
from text_retriever_class import TypesenseRetriever, TextRetriever

hybrid = TypesenseRetriever()
semantic = TextRetriever()

Typesense container started.



2024-04-07 16:11:00.943299: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-07 16:11:00.967788: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-07 16:11:01.430658: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [15]:
from tqdm import tqdm

ten_qas = qas.sample(n=1000).reset_index(drop=True)

In [16]:
hybrid_classifier_count = 0

for row in tqdm(ten_qas.iterrows()):
    retrieved = hybrid.retrieve(row[1]['question'], top_n=10).split("\n\n")
    df = data[data['chunked_data'].isin(retrieved)]

    if row[1]['url'] in df['url'].values:
        hybrid_classifier_count += 1

hybrid_classifier_count

1000it [07:45,  2.15it/s]


393

In [17]:
hybrid_no_classifier_count = 0

for row in tqdm(ten_qas.iterrows()):
    retrieved = hybrid.retrieve(row[1]['question'], top_n=10, use_classifier=False).split("\n\n")
    df = data[data['chunked_data'].isin(retrieved)]

    if row[1]['url'] in df['url'].values:
        hybrid_no_classifier_count += 1

hybrid_no_classifier_count

1000it [07:55,  2.10it/s]


478

In [18]:
rerank_classifier_count = 0

for row in tqdm(ten_qas.iterrows()):
    retrieved = semantic.retrieve(row[1]['question'], top_n=10).split("\n\n")
    df = data[data['chunked_data'].isin(retrieved)]

    if row[1]['url'] in df['url'].values:
        rerank_classifier_count += 1

rerank_classifier_count

1000it [04:12,  3.96it/s]


400

In [19]:
rerank_no_classifier_count = 0

for row in tqdm(ten_qas.iterrows()):
    retrieved = semantic.retrieve(row[1]['question'], top_n=10, use_classifier=False).split("\n\n")
    df = data[data['chunked_data'].isin(retrieved)]

    if row[1]['url'] in df['url'].values:
        rerank_no_classifier_count += 1

rerank_no_classifier_count

1000it [04:39,  3.58it/s]


481

In [20]:
semantic_classifier_count = 0

for row in tqdm(ten_qas.iterrows()):
    retrieved = semantic.retrieve(row[1]['question'], top_n=10, type="semantic").split("\n\n")
    df = data[data['chunked_data'].isin(retrieved)]

    if row[1]['url'] in df['url'].values:
        semantic_classifier_count += 1

semantic_classifier_count

1000it [00:26, 37.80it/s]


386

In [21]:
semantic_no_classifier_count = 0

for row in tqdm(ten_qas.iterrows()):
    retrieved = semantic.retrieve(row[1]['question'], top_n=10, type="semantic", use_classifier=False).split("\n\n")
    df = data[data['chunked_data'].isin(retrieved)]

    if row[1]['url'] in df['url'].values:
        semantic_no_classifier_count += 1

semantic_no_classifier_count

1000it [01:06, 14.99it/s]


486

In [22]:
print(f"Hybrid classifier count: {round(100*hybrid_classifier_count/len(ten_qas), 2)}%")
print(f"Hybrid no classifier count: {round(100*hybrid_no_classifier_count/len(ten_qas), 2)}%")
print(f"Rerank classifier count: {round(100*rerank_classifier_count/len(ten_qas), 2)}%")
print(f"Rerank no classifier count: {round(100*rerank_no_classifier_count/len(ten_qas), 2)}%")
print(f"Semantic classifier count: {round(100*semantic_classifier_count/len(ten_qas), 2)}%")
print(f"Semantic no classifier count: {round(100*semantic_no_classifier_count/len(ten_qas), 2)}%")

Hybrid classifier count: 39.3%
Hybrid no classifier count: 47.8%
Rerank classifier count: 40.0%
Rerank no classifier count: 48.1%
Semantic classifier count: 38.6%
Semantic no classifier count: 48.6%


From 1k TRAIN run:

Hybrid classifier count: 41.2%
Hybrid no classifier count: 45.4%
Rerank classifier count: 42.3%
Rerank no classifier count: 47.2%
Semantic classifier count: 39.6%
Semantic no classifier count: 45.8%

Hybrid classifier count: 47.0
Hybrid no classifier count: 57
Rerank classifier count: 44
Rerank no classifier count: 51
Semantic classifier count: 44
Semantic no classifier count: 53

In [43]:
%%timeit

retrieved = semantic.retrieve("Tell me about the math major at SUNY Brockport.", top_n=10, type="semantic", use_classifier=False)


73.1 ms ± 2.66 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [44]:
%%timeit

retrieved = semantic.retrieve("Tell me about the math major at SUNY Brockport.", top_n=10, type="semantic", use_classifier=True)

24.7 ms ± 5.41 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [36]:
%%timeit

retrieved = semantic.retrieve("Tell me about the math major at SUNY Brockport.", top_n=10, type="semantic_rerank", use_classifier=False)


276 ms ± 2.83 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [37]:
%%timeit

retrieved = semantic.retrieve("Tell me about the math major at SUNY Brockport.", top_n=10, type="semantic_rerank", use_classifier=True)

220 ms ± 3.88 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [48]:
%%timeit
retrieved = hybrid.retrieve("Tell me about the math major at SUNY Brockport.", top_n=10, use_classifier=False)

431 ms ± 64.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [47]:
%%timeit
retrieved = hybrid.retrieve("Tell me about the math major at SUNY Brockport.", top_n=10, use_classifier=True)

451 ms ± 74 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
